# Groeps Opdracht CI
## Joost Vledder, Sadjia Safdari, Simon Kreulen & Jasper van Eck

### Inhoudsopgave <a name='Top'></a>

[Imports](#import)

[Load data](#loaddata)

[Data Clean Up](#cleanup)

[Basic Data Stats](#basicdata)

[Plots](#plots)

[Basic Data Stats for Collabaritive Filtering](#basicdatacollab)

[Cosine Similiraty Content Based](#cossim)

[Query vector](#queryvector)

[SVD](#svd)

[K-Means](#kmeans)

[Cohen Kappa](#cohenkappa)

### Imports <a name='import'></a>

[Top](#Top)

In [1]:
import os
import pandas as pd
import numpy as np
np.seterr(divide='raise', over='raise', under='raise', invalid='raise')
import matplotlib.pyplot as plt
from collections import defaultdict
import json
#pip install geopy
from geopy import geocoders
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD

### Load data <a name='loaddata'></a>

[Top](#Top)

In [6]:
#rootdir = './yelp/data'
rootdir = './yelp/yelp/data'

df_business = pd.DataFrame()
df_users = pd.DataFrame()
df_reviews = pd.DataFrame()

count = 0

def load_jsons(data_path, file):
    # function to help load json files, since sometimes they give utf8 encoding errors, sometimes they don't
    file_path = os.path.join(subdir, file)
    lines = []
    with open(file_path) as jsons:
        try:
            lines = [json.loads(json_line) for json_line in jsons]
        except:
            print(file_path)
    return pd.DataFrame(lines)

for subdir, dirs, files in os.walk(rootdir):
    if count == 200: #1078 is the total amount of cities
        break
        
    for file in files:
        if os.path.basename(file) == 'business.json':
            df_business = df_business.append(load_jsons(subdir, file))
        if os.path.basename(file) == 'user.json':
            df_users = df_users.append(load_jsons(subdir, file))
        if os.path.basename(file) == 'review.json':
            df_reviews = df_reviews.append(load_jsons(subdir, file))
    count += 1

df_business = df_business.set_index('business_id')
df_users = df_users.set_index('user_id')
df_reviews = df_reviews.set_index('review_id')

display(df_business.head())
display(df_users.head())
display(df_reviews.head())

./yelp/yelp/data\110 las vegas\business.json
./yelp/yelp/data\110 las vegas\review.json
./yelp/yelp/data\110 las vegas\user.json
./yelp/yelp/data\agincourt\business.json
./yelp/yelp/data\agincourt\review.json
./yelp/yelp/data\agincourt\user.json
./yelp/yelp/data\ahwahtukee\business.json
./yelp/yelp/data\ahwahtukee\review.json
./yelp/yelp/data\ahwahtukee\user.json
./yelp/yelp/data\ahwatukee\business.json
./yelp/yelp/data\ahwatukee\review.json
./yelp/yelp/data\ahwatukee\user.json
./yelp/yelp/data\ahwatukee foothills village\business.json
./yelp/yelp/data\ahwatukee foothills village\review.json
./yelp/yelp/data\ahwatukee foothills village\user.json
./yelp/yelp/data\airdrie\business.json
./yelp/yelp/data\airdrie\review.json
./yelp/yelp/data\airdrie\user.json
./yelp/yelp/data\ajax\business.json
./yelp/yelp/data\ajax\review.json
./yelp/yelp/data\ajax\user.json
./yelp/yelp/data\akron\business.json
./yelp/yelp/data\akron\review.json
./yelp/yelp/data\akron\user.json
./yelp/yelp/data\alberta\bus

./yelp/yelp/data\bratenahl\business.json
./yelp/yelp/data\bratenahl\review.json
./yelp/yelp/data\bratenahl\user.json
./yelp/yelp/data\brecksville\business.json
./yelp/yelp/data\brecksville\review.json
./yelp/yelp/data\brecksville\user.json
./yelp/yelp/data\brentwood\business.json
./yelp/yelp/data\brentwood\review.json
./yelp/yelp/data\brentwood\user.json
./yelp/yelp/data\bridgeville\business.json
./yelp/yelp/data\bridgeville\review.json
./yelp/yelp/data\bridgeville\user.json
./yelp/yelp/data\broadlands\business.json
./yelp/yelp/data\broadlands\review.json
./yelp/yelp/data\broadlands\user.json
./yelp/yelp/data\broadview heights\business.json
./yelp/yelp/data\broadview heights\review.json
./yelp/yelp/data\broadview heights\user.json
./yelp/yelp/data\broadview hts\business.json
./yelp/yelp/data\broadview hts\review.json
./yelp/yelp/data\broadview hts\user.json
./yelp/yelp/data\brook park\business.json
./yelp/yelp/data\brook park\review.json
./yelp/yelp/data\brook park\user.json
./yelp/yel

./yelp/yelp/data\cheswick\user.json
./yelp/yelp/data\chomedey, laval\business.json
./yelp/yelp/data\chomedey, laval\review.json
./yelp/yelp/data\chomedey, laval\user.json
./yelp/yelp/data\church cove\business.json
./yelp/yelp/data\church cove\review.json
./yelp/yelp/data\church cove\user.json
./yelp/yelp/data\ch∩┐╜teauguay\business.json
./yelp/yelp/data\ch∩┐╜teauguay\review.json
./yelp/yelp/data\ch∩┐╜teauguay\user.json
./yelp/yelp/data\citibank\business.json
./yelp/yelp/data\citibank\review.json
./yelp/yelp/data\citibank\user.json
./yelp/yelp/data\city center\business.json
./yelp/yelp/data\city center\review.json
./yelp/yelp/data\city center\user.json
./yelp/yelp/data\clairton\business.json
./yelp/yelp/data\clairton\review.json
./yelp/yelp/data\clairton\user.json
./yelp/yelp/data\claremont\business.json
./yelp/yelp/data\claremont\review.json
./yelp/yelp/data\claremont\user.json
./yelp/yelp/data\clark\business.json
./yelp/yelp/data\clark\review.json
./yelp/yelp/data\clark\user.json
./ye

KeyError: "None of ['business_id'] are in the columns"

In [ ]:
df_business.to_pickle('./business_pickle.pkl')
df_users.to_pickle('./users_pickle.pkl')
df_reviews.to_pickle('./reviews_pickle.pkl')

In [ ]:
df_business = pd.read_pickle('./business_pickle.pkl')
df_users = pd.read_pickle('./users_pickle.pkl')
df_reviews = pd.read_pickle('./reviews_pickle.pkl')

### Clean up Data For Content Based <a name='cleanup'></a>

[Top](#Top)

In [ ]:
cat_dict = defaultdict(lambda: defaultdict(int))

# Drop na categories
df_business = df_business[df_business['categories'].notna()]

# Normalize long, lat, stars & review_count
#x = cos(lat) * cos(lon)
#y = cos(lat) * sin(lon)
df_business['stars'] = (df_business['stars']-df_business['stars'].min()) / (df_business['stars'].max()-df_business['stars'].min())
df_business['review_count'] = (df_business['review_count']-df_business['review_count'].min()) / (df_business['review_count'].max()-df_business['review_count'].min())
df_business['x_axis'] = np.cos(df_business['latitude']) * np.cos(df_business['longitude'])
df_business['y_axis'] = np.cos(df_business['latitude']) * np.sin(df_business['longitude'])

# Dataframe of categories one hot encoded per business_id
df_tmp = df_business['categories'].str.split(pat=', ',expand=True)

# Create dict of categories
for index, row in df_tmp.iterrows():
    for _,elem in row.items():
        if elem and elem not in cat_dict[index]:
            cat_dict[index][elem] += 1  

In [ ]:
# Create DF from dict
df_cats = pd.DataFrame.from_dict(cat_dict, orient='index')

#Drop NaN column
df_cats = df_cats.drop(columns='NaN',errors='ignore')

# Fillna with 0
df_cats = df_cats.fillna(0)

# Join back into business
df_business = df_business.join(df_cats,on='business_id')

# Drop cols
df_business = df_business.drop(columns=['address','city','state','postal_code','latitude','longitude','attributes','categories','hours'],errors='ignore')

# Remove elite & friends columns because of non use
df_users = df_users.drop(columns=['elite','friends'],axis=1,errors='ignore')

In [ ]:
display(df_business.head())

In [ ]:
df_business.to_pickle('./business_pickle.pkl')
df_users.to_pickle('./users_pickle.pkl')
df_reviews.to_pickle('./reviews_pickle.pkl')

In [ ]:
df_business = pd.read_pickle('./business_pickle.pkl')
df_users = pd.read_pickle('./users_pickle.pkl')
df_reviews = pd.read_pickle('./reviews_pickle.pkl')

### Basic Data Stats for Content Based <a name='basicdata'></a>

[Top](#Top)

In [ ]:
# Some information about the category amounts per business

# series of amount of categories per business
cats_business = df_cats.sum(axis=1)
display(cats_business)

avg_cats_bus = cats_business.mean()
max_cats_bus = cats_business.max()
min_cats_bus = cats_business.min()

print('Average amount of categories for businesses: ', avg_cats_bus)
print('Minimum amount of categories for businesses: ', min_cats_bus)
print('Maximum amount of categories for businesses: ', max_cats_bus)

# series of amount of businesses with the index amount of categories
cat_bus_distribution = cats_business.value_counts()
display(cat_bus_distribution)

In [ ]:
# series of number of businesses with index as category
cats_presences = df_cats.sum(axis=0).sort_values(ascending=False)
display(cats_presences)

avg_cats = cats_presences.mean()
max_cats = cats_presences.max()
min_cats = cats_presences.min()

### Plots <a name='plots'></a>

[Top](#Top)

In [ ]:
hist_bus = df_business.hist(column=['stars','review_count'])
df_business.plot(x='x_axis',y='y_axis',kind='scatter')
plt.show()
cats_presences.plot()

### Basic Data Stats for Collaborative Based <a name='basicdatacollab'></a>

[Top](#Top)

In [ ]:
# reset index
df_reviews = df_reviews.reset_index()

# count reviews per user and reviews per business
reviews_per_user = df_reviews.groupby("user_id")["review_id"].count().sort_values(ascending=False).to_frame()
reviews_per_business = df_reviews.groupby("business_id")["review_id"].count().sort_values(ascending=False).to_frame()


# plot both review counts
user_plot = reviews_per_user.plot()
business_plot = reviews_per_business.plot()

In [ ]:
reviews = df_reviews[:100]
reviews

In [ ]:
def pivot_ratings(ratings):
    pivottable = ratings.pivot_table(index="business_id", columns="user_id", values="stars").fillna(0)
    return pivottable

utility_matrix = pivot_ratings(reviews)
display(utility_matrix)

In [ ]:
def cosine_distance(matrix, id1, id2):
    selected_features = matrix.loc[id1].notna() & matrix.loc[id2].notna()
    
    # if no matching features, return 'not a number' (NaN)
    if not selected_features.any():
        return np.nan
    
    # get the features from the matrix
    features1 = matrix.loc[id1][selected_features]
    features2 = matrix.loc[id2][selected_features]
    
    if features1.equals(features2):
        return 1
    
    if all(x == 0 for x in features1):
        return np.nan
    if all(x == 0 for x in features2):
        return np.nan
    
    upper = sum(features1.multiply(features2, fill_value = 0))
    
    bottom1 = 0
    bottom2 = 0
    
    for i in range(len(features1)):
        bottom1 += list(features1)[i]**2
        bottom2 += list(features2)[i]**2
    bottom1 = bottom1**0.5
    bottom2 = bottom2**0.5
    
    bottom = bottom1 * bottom2
    
    distance = upper / bottom
    
    return distance
    
def create_similarity_matrix_cosine(matrix):
    similarity_matrix = pd.DataFrame(0, index=matrix.index, columns=matrix.index, dtype=float)
    for row in similarity_matrix:
        for column in similarity_matrix.index:
            similarity_matrix.at[column, row] = cosine_distance(matrix, row, column)
    
    return similarity_matrix

#display(create_similarity_matrix_euclid(matrix))
    
similarity = create_similarity_matrix_cosine(utility_matrix)
display(similarity)

In [ ]:
def mean_center_columns(matrix):
    return matrix.apply(lambda x: x - x.mean())

centered_utility_matrix = mean_center_columns(utility_matrix)
display(centered_utility_matrix)

similarity = create_similarity_matrix_cosine(centered_utility_matrix)
display(similarity)

In [ ]:
# TEST neighborhood op eentje

def select_neighborhood(similarities, ratings, k):
    """ selects all items with similarity > 0  """
    moviesNotWatched = ratings[ratings.isna()].index
    
    similarities = similarities.where(lambda x : x > 0.00).dropna()
    similarities = similarities.drop(labels = moviesNotWatched)
    
    return similarities
    

neighborhood = select_neighborhood(similarity["6E5SM3-YvBpdzNNW68ksTw"], utility_matrix["-1B9xAQxWufCB3ksliPxZA"], 10)
display(neighborhood)

In [ ]:
# TEST

def weighted_mean(neighborhood, ratings):  
    upper = 0
    bottom = 0
    
    for index in neighborhood.index:
        similarity = neighborhood.get(index)
        upper += ratings.get(index) * similarity
        bottom += similarity
    
    if bottom == 0:
        return np.nan
    else:
        return upper / bottom

neighborhood1 = select_neighborhood(similarity["6E5SM3-YvBpdzNNW68ksTw"], utility_matrix["-1B9xAQxWufCB3ksliPxZA"], 10)
prediction1 = weighted_mean(neighborhood1, utility_matrix["-1B9xAQxWufCB3ksliPxZA"])

neighborhood2 = select_neighborhood(similarity["UM07j7HtW-dhr8Gq3LQLxg"], utility_matrix["-1B9xAQxWufCB3ksliPxZA"], 10)
prediction2 = weighted_mean(neighborhood1, utility_matrix["-1B9xAQxWufCB3ksliPxZA"])

In [ ]:
"""""
userplot --> the distribution of the reviews over the users (long-tail): considering the fact that there are more
reviews on business than users (one user can rate more than 1 business), it is probably better to choose
item-based CF than user-based CF. 

"""""
users_series = df_users['review_count'].sort_values(ascending=False).drop_duplicates(keep='first')

display(users_series)
userplot = users_series.plot()

userplot.set(xlabel='user_id', ylabel='review_count')
plt.show()

In [ ]:
df_business = pd.read_pickle('./business_pickle.pkl')
df_cats = pd.read_pickle('./cats_pickle.pkl')
df_users = pd.read_pickle('./users_pickle.pkl')
df_reviews = pd.read_pickle('./reviews_pickle.pkl')

### Cosine Similarity <a name='cossim'></a>

[Top](#Top)

In [ ]:
# Input vectors are series, convert to np array
def cosineSim(vector, docVector):
    vector = vector.to_numpy()
    docVector = docVector.to_numpy()
    
    # Get lengths of vectors
    sim = (np.sqrt(sum(vector**2))*np.sqrt(sum(docVector**2)))
    
    # Ensure no division by 0
    if sim == 0.:
        return np.nan
    
    # Calculate cosine sim
    return vector.dot(docVector)/sim

In [ ]:
def rankedBusinessList(df_business, queryVector):
    df_bus_features = df_business.copy().drop(columns=['name','city','stars','review_count','is_open','Score Cos'],errors='ignore')
    scoreList = np.zeros(len(df_business))
    for i in range(len(df_bus_features)):
        scoreList[i] = cosineSim(queryVector, df_bus_features.iloc[i])
    
    df_business['Score Cos'] = scoreList
    return df_business.sort_values(by=['Score Cos','stars'],ascending=False)

### Create Query Vector <a name='queryvector'></a>

[Top](#Top)

This first cell get the longitude and latitude of a given city and state, and normalizes both values.

In [ ]:
loc_string = 'Las Vegas NV'
gn = Nominatim(user_agent='WalterGKurtz')
location = gn.geocode(loc_string)
queryVector = pd.Series(data=0.,index=df_business.columns)
queryVector.at['latitude'] = location.latitude
queryVector.at['longitude'] = location.longitude
#x = cos(lat) * cos(lon)
#y = cos(lat) * sin(lon)
#queryVector.at['x_axis'] = np.cos(location.latitude) * np.cos(location.longitude)
#queryVector.at['y_axis'] = np.cos(location.latitude) * np.sin(location.longitude)


The second cell inputs the preferences of the user.

In [ ]:
# Preferences
queryVector = queryVector.drop(labels=['name','city','stars','review_count','is_open'],errors='ignore')
queryVector.at['Automotive'] = 0
queryVector.at['Auto Repair'] = 0
queryVector.at['Fast Food'] = 0
queryVector.at['Restaurants'] = 0
queryVector.at['Pizza'] = 1.
queryVector.at['Men\'s Clothing'] = 0
queryVector.at['Women\'s Clothing'] = 0
queryVector.at['Fashion'] = 0
queryVector.at['Bars'] = 0
display(queryVector)
#queryVector.to_csv('test.csv', header=False)

In [ ]:
display(rankedBusinessList(df_business,queryVector).head(20))

### SVD <a name='svd'></a>

[Top](#Top)

In [ ]:
stars = df_business['stars'].values
cities = df_business['city'].values
names = df_business['name'].values
bus_ids = df_business.index
df_tmp_svd = df_business.copy().drop(columns=['name','city', 'stars','review_count','is_open', 'Score Cos'],errors='ignore').to_numpy()

In [ ]:
# This code can be used to find the optimal amount of dimension by the SVD

# trunc_svd = TruncatedSVD(n_components=len(df_tmp_svd[0])-1).fit(df_tmp_svd)
# reduced_x = trunc_svd.transform(df_tmp_svd)
# singles = trunc_svd.singular_values_

# eigvals = singles**2 / np.sum(singles**2)
# fig = plt.figure(figsize=(8,5))
# sing_vals = np.arange(len(df_tmp_svd[0])-1) + 1
# plt.plot(sing_vals[2:200], eigvals[2:200], 'ro-', linewidth=2)
# plt.title('Scree Plot')
# plt.xlabel('Component number')
# plt.ylabel('Eigenvalue')
# plt.show()

In [ ]:
optimal_dims = 25
trunc_svd = TruncatedSVD(n_components=optimal_dims).fit(df_tmp_svd)
reduced_x = trunc_svd.transform(df_tmp_svd)

In [ ]:
queryVectorReduced = trunc_svd.transform(np.array([queryVector]))

df_reduced_x = pd.DataFrame(data=reduced_x, index=bus_ids)
df_reduced_x.insert(0, 'stars', stars)
df_reduced_x.insert(0, 'city', cities)
df_reduced_x.insert(0, 'name', names)
df_queryVectorReduced = pd.Series(data=queryVectorReduced[0])
display(rankedBusinessList(df_reduced_x,df_queryVectorReduced).head(20))

### K-means <a name='kmeans'></a>

[Top](#Top)

In [ ]:
# # This block of code was used to find the optimal K, it takes very long to run, so that's why it is commented now.

# from yellowbrick.cluster import KElbowVisualizer

# df_bus_kmeans = df_reduced_x.copy().drop(columns=['name','city','stars','Score Cos'],errors='ignore').to_numpy()

# # choose k

# model = KMeans(random_state=0)
# visualizer = KElbowVisualizer(model, k=[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
# visualizer.fit(df_bus_kmeans)        # Fit the data to the visualizer
# visualizer.show() 

In [ ]:
df_bus_kmeans = df_reduced_x.copy().drop(columns=['name','city','stars','Score Cos'],errors='ignore').to_numpy()
kmeans = KMeans(n_clusters=300,random_state=0).fit(df_bus_kmeans)

In [ ]:
# Predict query vector
labels = np.array(kmeans.predict(df_bus_kmeans))
prediction = kmeans.predict([df_queryVectorReduced.to_numpy()])

matches = [y[0] for y,val in np.ndenumerate(labels) if val == prediction]

df_matched = df_business.iloc[matches]

display(df_matched.sort_values(by=['stars'],ascending=False).head(20))

### Cohen's Kappa <a name='cohenkappa'></a>

[Top](#Top)

In [ ]:
# Dataformat
# 1 | 0
# 1 | 1
# 0 | 1

def PE(data):
    '''On input data, return the P(E) (expected agreement).'''
    relevant = 0
    nonrelevant = 0
    # Iterate over the data
    for i in data:
        for j in i:
            
            # Top up the relevant documents by one if 1 is encountered
            if j == 1:
                relevant += 1
            # Top up the nonrelevant documents by one if 0 is encountered
            if j == 0:
                nonrelevant += 1

    # Calculates the total of inspected documents for the judges combined
    total = len(data)*2

    # Calculates the pooled marginals
    rel = relevant/total
    nonrel = nonrelevant/total

    # Calculates the P(E)
    P_E = nonrel**2 + rel **2    
    return    P_E 


def kappa(data, P_E):
    agree = 0
    for i in data:
        temp = None
        for j in i:
            if temp == j:
                agree += 1
            temp = j
    P_A = agree / len(data)
    if P_E == 1:
        kappa = 1
    else:
        kappa = (P_A - P_E)/(1 - P_E)   
    return kappa

In [ ]:
def AveragePrecision(ranked_list_of_results, list_of_relevant_objects):
    total = len(list_of_relevant_objects)
    sumPk = 0
    rank = 0
    relevant = 0
    for result in ranked_list_of_results:
        rank += 1
        if result in list_of_relevant_objects:
            relevant += 1
            sumPk += relevant/rank
            
    aprecision = sumPk/total
    return aprecision